# Using Microsoft Azure OpenAI and SQL Server Machine Learning Service

### Proof-of-Concept Demonstration Project

_Buck Woody, Principal Applied Data Scientist, Microsoft - Last edited 06/07/2023_

_These sample scripts are provided <span style="color:#111111">AS IS without warranty of any kind. Microsoft further disclaims all implied warranties</span> including, without limitation, any implied warranties of merchantability or of fitness for a particular purpose. The entire risk arising out of the use or performance of the sample scripts and documentation remains with you._

Business Scenarios for AdventureWorks are here: https:\learn.microsoft.com\en-us\previous-versions\sql\sql-server-2008\ms124825(v=sql.100) 

Restore the _AdventureWorks_ sample database suitable for your installation version. This example uses SQL Server 2019 Machine Learning Services in a Container, where the _AdventureWorks_ backup file has been copied to the Container.

In [26]:
USE master;
GO
RESTORE DATABASE [AdventureWorks]
FROM DISK = '/var/opt/mssql/data/AdventureWorks2019.bak'
WITH MOVE 'AdventureWorks2019' TO '/var/opt/mssql/data/AdventureWorks_Data.mdf'
, MOVE 'AdventureWorks2019_Log' TO '/var/opt/mssql/log/AdventureWorks_Log.ldf'
, RECOVERY, REPLACE, STATS = 10;

Commands completed successfully.

Total execution time: 00:00:00.002

Check to ensure Machine Learning Services are installed, turn on if not configured. Also ensure the _SQLLaunchpad_ Service is enabled and started in _SQL Server Configuration Manager_. 

You can learn more about installing SQL Server Machine Learning Services here: https:\learn.microsoft.com\en-us\sql\machine-learning\install\sql-machine-learning-services-windows-install?view=sql-server-ver15&viewFallbackFrom=sql-server-ver16

In [1]:
/* Check to see if the feature is enabled... */
EXEC sp_configure 'clr enabled'

/* If 0, then */
EXEC sp_configure 'clr enabled', 1
EXEC sp_configure 'external scripts enabled', 1;
RECONFIGURE WITH OVERRIDE; 

/* A quick text return from Python combining the input data with a string */
EXECUTE sp_execute_external_script @language = N'Python'
    , @input_data_1 = N'SELECT ''Hello'''
    , @input_data_1_name = N'testvalue'
    , @script = N'OutputDataSet = testvalue + '' there, General Kenobi...'''
WITH RESULT SETS(([Grievous] VARCHAR(MAX) NOT NULL));

Configuration option 'clr enabled' changed from 1 to 1. Run the RECONFIGURE statement to install.

Configuration option 'external scripts enabled' changed from 1 to 1. Run the RECONFIGURE statement to install.

(1 row affected)

Total execution time: 00:00:03.151

name,minimum,maximum,config_value,run_value
clr enabled,0,1,1,1


Grievous
"Hello there, General Kenobi..."


Use Microsoft Azure OpenAI Studio to obtain the _api\_base_, _api\_type_, _api\_version_, and _api\_key_ values from your deployment. More on creating a Microsoft Azure OpenAI Deployment is here: https:\learn.microsoft.com\en-us\azure\cognitive-services\openai\how-to\create-resource?pivots=web-portal 

**NOTE**: _This data should be in a restricted table, or encrypted at the row or value level to ensure the highest security. Alternatively, you could store the data in Microsoft Azure Vault, and use the Python code that follows later to retrieve those values.

In [ ]:
USE AdventureWorks; 
GO 

INSERT INTO OpenAIData (api_base, api_type, api_version, api_key)
VALUES ('<Your API Base Value Here>'
, '<Your API Type Value Here>'
, '<Your API Version Value Here>'
, '<Your API Key Value Here>');
GO

SQL Server Machine Learning Services takes one input variable to the script (in this case Python), so we need to "flatten" the values for our connection information and the item we want to query. We'll experiment a bit here to get that into a single string.

In [4]:
USE AdventureWorks;
GO

/* Show product and description as single string */
SELECT Name  + ': ' + Description
FROM production.vProductAndDescription
WHERE ProductID = 980 and CultureID = 'en';
GO

/* Show the Azure OpenAI  API Values */
SELECT api_base, api_type, api_version, api_key
FROM OpenAIData;
GO

/* Test to get proper output */
SELECT api_base, api_type, api_version, api_key,
(SELECT Name  + ': ' + Description FROM production.vProductAndDescription WHERE ProductID = 980 and CultureID = 'en')
FROM OpenAIData;
GO

Commands completed successfully.

(1 row affected)

(1 row affected)

(1 row affected)

Total execution time: 00:00:00.104

(No column name)
"Mountain-400-W Silver, 38: This bike delivers a high-level of performance on a budget. It is responsive and maneuverable, and offers peace-of-mind when you decide to go off-road."


api_base,api_type,api_version,api_key
<Your API Base Value Here>,<Your API Type Value Here>,<Your API Version Value Here>,<Your API Key Value Here>


api_base,api_type,api_version,api_key,(No column name)
<Your API Base Value Here>,<Your API Type Value Here>,<Your API Version Value Here>,<Your API Key Value Here>,"Mountain-400-W Silver, 38: This bike delivers a high-level of performance on a budget. It is responsive and maneuverable, and offers peace-of-mind when you decide to go off-road."


After a bit of experimentation with Python, we find that SQL Server Machine Learning Services takes in a single string as a variable as a Pandas DataFrame object. We construct the string as per the previous experiment, and then add the libraries to work with that string in Pandas. 

Using the _iloc_ Pandas function and grabbing the indexes of the expected values, we create a combined, secure set of values, including the api and key needed by the Azure OpenAI deployment. 

We construct the prompt (the _FullPrompt_ variable) using the general instructions for the prompt we want to send, and inject the product name and the short description to get as much fidelity as we can from Azure OpenAI.

We call to Azure OpenAI and store the result in an object. We want only the content of the response from that object, so we convert that portion of the object <span style="color: rgb(163, 21, 21); font-family: Consolas, &quot;Courier New&quot;, monospace; font-size: 12px; white-space: pre;">response[''choices''][0][''message''][''content''] </span> to a string. 

SQL Server Machine Learning Services returns data from Python as another Pandas DataFrame object, so we take the string and convert it to a DataFrame, catching that output into a VARCHAR(MAX) column.

In [2]:
USE AdventureWorks;
GO

/* Final Query */
DECLARE @sqltext nvarchar(max)

SET @sqltext = '
SELECT api_base, api_type, api_version, api_key,
(SELECT Name  + '': '' + Description FROM production.vProductAndDescription WHERE ProductID = 980 and CultureID = ''en'' )  AS ''Description'' 
FROM OpenAIData
'
EXECUTE sp_execute_external_script @language = N'Python'
, @input_data_1 = @sqltext
, @input_data_1_name = N'InputText'
,  @script=N'
import openai
import pandas as pd
import json
from io import StringIO

InputStrings = pd.DataFrame(InputText)

openai.api_base = str(InputStrings.iloc[0, 0])
openai.api_type = str(InputStrings.iloc[0, 1])
openai.api_version = str(InputStrings.iloc[0, 2])
openai.api_key = str(InputStrings.iloc[0, 3])
Description = str(InputStrings.iloc[0, 4])

FullPrompt = ''You are an experienced marketing expert. Generate 2000 letters of ad copy for '' +str(Description)+ ''. Do not add testimonials or a signature block.''

response = openai.ChatCompletion.create(
  engine="bwoody-turbo",
    messages = [{"role":"system","content":FullPrompt},
  {"role":"user","content":""},{"role":"assistant","content":""},{"role":"user","content":FullPrompt}],
  temperature=0.5,
  max_tokens=800,
  top_p=0.95,
  frequency_penalty=0,
  presence_penalty=0,
  stop=None)

AdCopy = str(response[''choices''][0][''message''][''content'']) 

OutputDataSet = pd.DataFrame([AdCopy])
'
WITH RESULT SETS(([AdCopy] VARCHAR(MAX)));
GO

(1 row affected)

Total execution time: 00:00:09.256

AdCopy
"Dear Mountain Bike Enthusiast, Are you looking for a bike that can deliver the performance you need without breaking the bank? Look no further than the Mountain-400-W Silver, 38. This bike is designed to give you the high-level performance you need to tackle any terrain, without the high price tag. With its responsive and maneuverable design, you'll be able to navigate even the toughest trails with ease. But it's not just about performance - the Mountain-400-W Silver, 38 also offers peace-of-mind when you decide to go off-road. With its sturdy construction and reliable components, you can trust this bike to handle anything you throw at it. So why wait? Order your Mountain-400-W Silver, 38 today and experience the ultimate in off-road performance at a price you can afford. Don't settle for anything less than the best - choose the Mountain-400-W Silver, 38 and take your mountain biking to the next level. Sincerely, [Your Name]"


Our Proof-of-Concept is complete. This last section of code should be wrapped in a Stored Procedure with an input variable that takes the product number from the query, and the Stored Procedure should follow standard chain-of-permission security for our organization. 

  

The output can be stored directly into the table, but a better practice would be to present the text to the end user for editing and proofing, since Generative Pre-trained Transformer text may not be correct, approriate or exact. After the user edits the values, the application can safely store the result.